In [1]:
import org.apache.spark

Intitializing Scala interpreter ...

Spark Web UI available at http://10.201.102.252:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1737387180326)
SparkSession available as 'spark'


import org.apache.spark


In [ ]:
val sc = spark.SparkContext.getOrCreate()

In [43]:
val path_to_datasets = "../../../datasets/processed"

val path_to_tracks = path_to_datasets + "/tracks.csv"
val path_to_playlists = path_to_datasets + "/playlists.csv"
val path_to_track_in_playlists = path_to_datasets + "/tracks_in_playlist.csv"
val path_to_artists = path_to_datasets + "/artists.csv"

path_to_datasets: String = ../../../datasets/processed
path_to_tracks: String = ../../../datasets/processed/tracks.csv
path_to_playlists: String = ../../../datasets/processed/playlists.csv
path_to_track_in_playlists: String = ../../../datasets/processed/tracks_in_playlist.csv
path_to_artists: String = ../../../datasets/processed/artists.csv


In [44]:
object CsvParser {

  val noGenresListed = "(no genres listed)"
  val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
  val quotes = "\""

  //  case class PlayList(PID: String, playlist_name: String, num_followers: Int)
  //
  //  case class Track(track_uri: String, track_name: String, duration_ms: Int, artist_uri: String, album_uri: String, album_name: String)
  //
  //  case class Artist(artist_uri: String, artist_name: String)
  //
  //  case class TrackInPlaylist(PID: String, track_uri: String, pos: Int)

  //  // (PID, playlist_name, num_followers)
  //  def parsePlayListLine(line: String): Option[PlayList] = {
  //    try {
  //      val input = line.split(commaRegex)
  //      Some(PlayList(input(0).trim, input(1).trim, input(2).trim.toInt))
  //    } catch {
  //      case _: Exception => None
  //    }
  //  }
  //
  //  // (track_uri, track_name, duration_ms, artist_uri, album_uri, album_name)
  //  def parseTrackLine(line: String): Option[Track] = {
  //    try {
  //      val input = line.split(commaRegex)
  //      Some(Track(input(0).trim, input(1).trim, input(2).trim.toInt, input(3).trim, input(4).trim, input(5).trim))
  //    } catch {
  //      case _: Exception => None
  //    }
  //  }
  //
  //  // (artist_uri, artist_name)
  //  def parseArtistLine(line: String): Option[Artist] = {
  //    try {
  //      val input = line.split(commaRegex)
  //      Some(Artist(input(0).trim, input(1).trim))
  //    } catch {
  //      case _: Exception => None
  //    }
  //  }
  //
  //  // (PID, track_uri, pos)
  //  def parseTrackInPlaylistLine(line: String): Option[TrackInPlaylist] = {
  //    try {
  //      val input = line.split(commaRegex)
  //      Some(TrackInPlaylist(input(0).trim, input(1).trim, input(2).trim.toInt))
  //    } catch {
  //      case _: Exception => None
  //    }
  //  }
  // (PID, playlist_name, num_followers)
  def parsePlayListLine(line: String): Option[(String, String, Int)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim, input(2).trim.toInt)
    } catch {
      case _: Exception => None
    }
  }

  // (track_uri, track_name, duration_ms, artist_uri, album_uri, album_name)
  def parseTrackLine(line: String): Option[(String, String, Int, String, String, String)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim, input(2).trim.toInt, input(3).trim, input(4).trim, input(5).trim)
    } catch {
      case _: Exception => None
    }
  }

  // (artist_uri, artist_name)
  def parseArtistLine(line: String): Option[(String, String)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim)
    } catch {
      case _: Exception => None
    }
  }

  // (PID, track_uri, pos)
  def parseTrackInPlaylistLine(line: String): Option[(String, String, Int)] = {
    try {
      val input = line.split(commaRegex)
      Some(input(0).trim, input(1).trim, input(2).trim.toInt)
    } catch {
      case _: Exception => None
    }
  }
}

defined object CsvParser


In [45]:
val rddTracks = sc.textFile(path_to_tracks).
  flatMap(CsvParser.parseTrackLine)

val rddPlaylists = sc.textFile(path_to_playlists).
  flatMap(CsvParser.parsePlayListLine)

val rddTrackInPlaylists = sc.textFile(path_to_track_in_playlists).
  flatMap(CsvParser.parseTrackInPlaylistLine)

val rddArtists = sc.textFile(path_to_artists).
  flatMap(CsvParser.parseArtistLine)

rddTracks: org.apache.spark.rdd.RDD[(String, String, Int, String, String, String)] = MapPartitionsRDD[427] at flatMap at <console>:31
rddPlaylists: org.apache.spark.rdd.RDD[(String, String, Int)] = MapPartitionsRDD[430] at flatMap at <console>:34
rddTrackInPlaylists: org.apache.spark.rdd.RDD[(String, String, Int)] = MapPartitionsRDD[433] at flatMap at <console>:37
rddArtists: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[436] at flatMap at <console>:40


In [46]:
// Join all the RDDs
val rddPlaylistTracksInPlaylist = rddPlaylists.keyBy(_._1)
    .join(rddTrackInPlaylists.keyBy(_._1))
    .map(x => (x._2._2._1, x._2._2._2))

val rddPlaylistTracksInPlaylistTracks = rddPlaylistTracksInPlaylist.keyBy(_._2)
  .join(rddTracks.keyBy(_._1))
  // take all fields of the track, and the playlist PID
  .map(x => (x._2._1._1, x._2._2._2, x._2._2._3, x._2._2._4, x._2._2._5, x._2._2._6))
val rddPlaylistTracksInPlaylistTracksArtists = rddPlaylistTracksInPlaylistTracks.keyBy(_._4)
  .join(rddArtists.keyBy(_._1))
  // keep all the fields of the track, and the playlist PID and the artist name
  .map(x => (x._2._1._1, x._2._1._2, x._2._1._3, x._2._1._4, x._2._1._5, x._2._1._6, x._2._2._2))



rddPlaylistTracksInPlaylist: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[442] at map at <console>:33
rddPlaylistTracksInPlaylistTracks: org.apache.spark.rdd.RDD[(String, String, Int, String, String, String)] = MapPartitionsRDD[448] at map at <console>:38
rddPlaylistTracksInPlaylistTracksArtists: org.apache.spark.rdd.RDD[(String, String, Int, String, String, String, String)] = MapPartitionsRDD[454] at map at <console>:42


In [48]:
// (PID, track_name, duration_ms, artist_uri, album_uri, album_name, artist_name)
val pidArtistTrack = rddPlaylistTracksInPlaylistTracksArtists.map(x => ((x._1, x._4), 1))
val artistTrackCount = pidArtistTrack
  .reduceByKey(_ + _)

val pidToArtistTracks = artistTrackCount.map(x => (x._1._1, x._2))
val averageSongsPerArtist = pidToArtistTracks
  .groupByKey() // Raggruppa tutte le playlist
  .mapValues { counts =>
    val totalArtists = counts.size
    val totalTracks = counts.sum
    totalTracks.toDouble / totalArtists
  }

//averageSongsPerArtist.collect().foreach { case x =>
// println(s"Playlist ${x._1} ha una media di ${x._2} canzoni per artista.")
//}

// Step 6: Calcola la media complessiva dei valori medi
val totalPlaylists = averageSongsPerArtist.count() // Numero totale di playlist
val sumOfAverages = averageSongsPerArtist.map(_._2).sum() // Somma di tutte le medie

val overallAverage = sumOfAverages / totalPlaylists // Media complessiva

// Mostra il risultato
println(s"La media complessiva di canzoni per artista è: $overallAverage")


La media complessiva di canzoni per artista è: 2.1668235394982664


pidArtistTrack: org.apache.spark.rdd.RDD[((String, String), Int)] = MapPartitionsRDD[461] at map at <console>:33
artistTrackCount: org.apache.spark.rdd.RDD[((String, String), Int)] = ShuffledRDD[462] at reduceByKey at <console>:35
pidToArtistTracks: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[463] at map at <console>:37
averageSongsPerArtist: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[465] at mapValues at <console>:40
totalPlaylists: Long = 198708
sumOfAverages: Double = 430565.1718866215
overallAverage: Double = 2.1668235394982664
